In [2]:
import pickle

In [1]:
import pandas as pd

In [25]:
pip install tqdm

     |████████████████████████████████| 59 kB 7.8 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
from tqdm import tnrange, notebook,tqdm_notebook

In [3]:
infile = open('20200106_AISTripMainData.pkl','rb')
TripMain = pickle.load(infile)
infile.close()

In [4]:
infile = open('20200106_AISTripStatsData.pkl','rb')
TripStat = pickle.load(infile)
infile.close()

In [5]:
TripMain.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,Speed,point_id,Stationary,TimeCondition,medianSpeed,zScoreSpeed,Cos,trip_id
17394,636626,2017-07-31 16:52:06,37.16915,-76.60553,2.8,173.3,511.0,True,90.0,0.126114,5.044564,17394,True,1.323529,32.054958,-0.896133,0.000000,17277
17395,636626,2017-07-31 16:54:06,37.16767,-76.60873,5.3,-171.8,511.0,True,120.0,0.328257,9.847713,17395,False,1.764706,32.054958,-0.736777,0.999898,17277
17396,636626,2017-07-31 16:56:36,37.16584,-76.61284,5.3,-159.4,511.0,True,150.0,0.417732,10.025564,17396,False,2.205882,32.054958,-0.730877,0.955376,17277
17397,636626,2017-07-31 16:58:15,37.16548,-76.61585,5.4,-139.5,511.0,True,99.0,0.270312,9.829515,17397,False,1.455882,32.054958,-0.737381,0.998304,17277
17398,636626,2017-07-31 16:59:26,37.16534,-76.61815,6.4,-144.0,511.0,True,71.0,0.204873,10.387903,17398,False,1.044118,32.054958,-0.718855,0.998649,17277


In [6]:
TripStat.head()

,trip_id,MMSI,Dist,TimeDiff,Count,Speed,Cos
0,17277,636626,17.825440,1692.0,23,37.926467,0.889481
1,18037,641114,5.269649,1785.0,23,10.627863,0.905487
2,18220,641114,28.783308,9488.0,142,10.921154,0.940368
3,18222,641114,145.076523,38449.0,550,13.583591,0.978832
4,18502,641114,10.048207,3120.0,44,11.594085,0.942711


In [7]:
TripStat.columns

Index(['trip_id', 'MMSI', 'Dist', 'TimeDiff', 'Count', 'Speed', 'Cos'], dtype='object')

###  select trips in trip dataset

In [9]:
output = TripMain[TripMain['trip_id'].isin(TripStat['trip_id'].unique())]
output.to_csv('merged.csv',index=False)

In [37]:
# output = TripMain.merge(TripStat[['trip_id']],on=['trip_id'],how='left')
# output.to_csv('merged.csv',index=False)

In [2]:
output = pd.read_csv('merged.csv',parse_dates=['BaseDateTime'])
output = output.dropna(subset=['BaseDateTime','LAT','LON','trip_id'])

In [3]:

def sampleLat(df,OriginIndex,OriginLAT,OriginLON,TimeCount):
    if df['inserted'] != False:
        insertedIndex = df['Index']
        if insertedIndex > 0:
            wholeIndex = OriginIndex + [insertedIndex]
            wholeIndex.sort()
            location = wholeIndex.index(insertedIndex)
#             print(location)
            Lat0 = OriginLAT[location-1]
            Lon0 = OriginLON[location-1]
            Lat1 = OriginLAT[location]
            Lon1 = OriginLON[location]
            timeDelta = TimeCount[location] - TimeCount[location-1] 
            currentTime = df['timeCount']
            timeGap = currentTime - TimeCount[location-1]
            lat = ((Lat1 - Lat0) / timeDelta*timeGap) + Lat0
            return lat
                
        else:
            return df['LAT']
    else:
        return df['LAT']
def sampleLon(df,OriginIndex,OriginLAT,OriginLON,TimeCount):
    if df['inserted'] != False:
        insertedIndex = df['Index']
        if insertedIndex > 0:
            wholeIndex = OriginIndex + [insertedIndex]
            wholeIndex.sort()
            location = wholeIndex.index(insertedIndex)
#             print(location)
            Lat0 = OriginLAT[location-1]
            Lon0 = OriginLON[location-1]
            Lat1 = OriginLAT[location]
            Lon1 = OriginLON[location]
            timeDelta = TimeCount[location] - TimeCount[location-1] 
            currentTime = df['timeCount']
            timeGap = currentTime - TimeCount[location-1]
            lon = (Lon1 - Lon0) / timeDelta*timeGap + Lon0
#             print(location)

            return lon
        else:
            return df['LON']
    else:
        return df['LON']

In [4]:
rawData = output[['MMSI', 'BaseDateTime', 'LAT', 'LON', 'trip_id']]

In [12]:
trip = output['trip_id'].unique().tolist()[0]
# _ = [(lambda x: x+1)(i) for i in notebook.tqdm(outputut['trip_id'].unique().tolist()[1:])]

outputSample = rawData[rawData['trip_id']==trip]
outputSample = outputSample.dropna()
outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                      - outputSample.iloc[0,1]).seconds)
count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
#     TimeDelta['inserted'] = True
TimeDelta['timeCount'] = [60*i for i in range(count+1)]
outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
outputSample['inserted'] = False
outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])
outputSample = outputSample.reset_index()
del outputSample['index']
outputSample['Index'] = outputSample.index
outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
outputSample.loc[outputSample['inserted']==False,'timeCount_y'] = 0
outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
del outputSample['timeCount_y']
del outputSample['timeCount_x']
OriginIndex = outputSample[outputSample['inserted']==False]['Index'].tolist()
OriginLAT = outputSample[outputSample['inserted']==False]['LAT'].tolist()
OriginLON = outputSample[outputSample['inserted']==False]['LON'].tolist()
TimeCount = outputSample[outputSample['inserted']==False]['timeCount'].tolist()
outputSample['LAT'] = outputSample.apply(sampleLat, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
outputSample['LON'] = outputSample.apply(sampleLon, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
MMSI = outputSample.iloc[0,0]
tripID = outputSample['trip_id'].unique().tolist()[0]
# print(tripID)
outputSample['MMSI'] = outputSample['MMSI'].fillna(MMSI)
outputSample['trip_id'] = outputSample['trip_id'].fillna(tripID)
f = open("doneTrip.csv", "w")
f.write('trip'+'\n')
f.write(str(outputSample.trip_id.unique().tolist()[0])+'\n')
f.close()
result = outputSample[outputSample['BaseDateTime'].isin(timeDelta)]
result = result[['MMSI','trip_id','BaseDateTime','LAT','LON']]
result.to_csv('interpolated.csv',index=False)

In [13]:
doneTrip = pd.read_csv('doneTrip.csv')
doneTrip.head()

,trip
0,17277.0


In [ ]:
pbar = notebook.tqdm(total=len(output['trip_id'].unique().tolist()[1:]))
for trip in output['trip_id'].unique().tolist()[1:]:
    pbar.update(1)
    if trip in doneTrip.trip.unique().tolist():
        pass
    else:
        outputSample = rawData[rawData['trip_id']==trip]
        outputSample = outputSample.dropna()
        outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                              - outputSample.iloc[0,1]).seconds)
        count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
        timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
        TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
    #     TimeDelta['inserted'] = True
        TimeDelta['timeCount'] = [60*i for i in range(count+1)]
        outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
        outputSample['inserted'] = False
        outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])
        outputSample = outputSample.reset_index()
        del outputSample['index']
        outputSample['Index'] = outputSample.index
        outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
        outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
        outputSample.loc[outputSample['inserted']==False,'timeCount_y'] = 0
        outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
    #     outputSample['inserted'] = outputSample['inserted_x'] * outputSample['inserted_y']
        del outputSample['timeCount_y']
        del outputSample['timeCount_x']
        OriginIndex = outputSample[outputSample['inserted']==False]['Index'].tolist()
        OriginLAT = outputSample[outputSample['inserted']==False]['LAT'].tolist()
        OriginLON = outputSample[outputSample['inserted']==False]['LON'].tolist()
        TimeCount = outputSample[outputSample['inserted']==False]['timeCount'].tolist()
        outputSample['LAT'] = outputSample.apply(sampleLat, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
        outputSample['LON'] = outputSample.apply(sampleLon, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
        MMSI = outputSample.iloc[0,0]
        tripID = outputSample['trip_id'].unique().tolist()[0]
        outputSample['MMSI'] = outputSample['MMSI'].fillna(MMSI)
        outputSample['trip_id'] = outputSample['trip_id'].fillna(tripID)
        f = open("doneTrip.csv", "w")
#         f.write('trip'+'\n')
        f.write(str(outputSample.trip_id.unique().tolist()[0])+'\n')
        f.close()
        result = outputSample[outputSample['BaseDateTime'].isin(timeDelta)]
        result = result[['MMSI','trip_id','BaseDateTime','LAT','LON']]
        result.to_csv('interpolated.csv',index=False)